# Support Vector Machine for topic classification
---

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import json

In [2]:
module_path = os.path.abspath(os.path.join('..\..')) # Path to root folder
if module_path not in sys.path:
    sys.path.append(module_path + "/scripts") # define scripts path

from ipynb_func import *

Data loader:

In [3]:
#NUM = 10 # Number of data parquets to use
#assert NUM >= 1 and NUM <= 10, "NUM value must be in range [1, 10]"

# Making list of roots to merge processed raw data 
#paths = [module_path + f"/data/pikabu/tag_processed/raw_data/{i}_tag_processed.parquet" for i in range(NUM)] 

# Making list of roots to merge processed filtered data
#paths = [module_path + f"/data/pikabu/tag_processed/filtered_data/{i}_tag_processed.parquet" for i in range(NUM)] 

# Making list of roots to merge processed cleared data
paths = [module_path + f"/data/pikabu/splited_data/cleared_texts.parquet"] 

data = merge_dataset(paths)

In [4]:
pd.set_option('display.max_colwidth', 130)
data.head(3)

,id,text_markdown,tags
15,6991359,"[добрый, сутки, господин, дама, подсказывать, название, игра, телефон, оформление, убийство, зомби, очки, ездить, машинка, кру...","[игры, поиск]"
37,7004423,"[ехать, девчонка, школа, оставаться, свободный, макс, заявка, прямой, конечный, адрес, железнодорожный, институт, включать, вб...",[юмор]
52,6991603,"[стадо, стадо, гигантский, случаться, стадо, управлять, волк, предел, волк, жопа, враг, дружно, осматривать, выдавливать, стад...",[мат]


---
# 1. Data preparation and split

In [5]:
with open(module_path + f"/data/pikabu/splited_data/indexes.json") as f:
    id_splits = f.read()

id_splits = json.loads(id_splits)

data_train = data[data['id'].isin(id_splits['train'])]
data_val = data[data['id'].isin(id_splits['val'])]
data_test = data[data['id'].isin(id_splits['test'])]

In [6]:
print(f"Number of train data: {len(data_train)}")
print(f"Number of val data: {len(data_val)}")
print(f"Number of test data: {len(data_test)}")
print(f"Distribution: {len(data_train)/len(data)*100:.0f} / {len(data_val)/len(data)*100:.0f} / {len(data_test)/len(data)*100:.0f}")

Number of train data: 25209
Number of val data: 2821
Number of test data: 3119
Distribution: 81 / 9 / 10


---
# 2. Model training

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

from joblib import dump, load

Target preparation:

In [8]:
Vec = CountVectorizer(tokenizer=lambda x: x.split(','), binary=True)

df = data.copy()
df.tags = [','.join(i) for i in df.tags]

df_train = data_train.copy()
df_train.tags = [','.join(i) for i in df_train.tags]

df_val = data_val.copy()
df_val.tags = [','.join(i) for i in df_val.tags]

df_test = data_test.copy()
df_test.tags = [','.join(i) for i in df_test.tags]

y_data = Vec.fit(df['tags'])
y_train = Vec.transform(df_train['tags'])
y_val = Vec.transform(df_val['tags'])
y_test = Vec.transform(df_test['tags'])

c:\Users\decique\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
print('Tags to predict:')
print(Vec.get_feature_names_out())

Tags to predict:
['авто' 'авторский рассказ' 'алкоголь' 'анекдот' 'армия' 'вопрос' 'врачи'
 'девушки' 'деньги' 'дети' 'детство' 'другое' 'жизнь' 'игры' 'интересное'
 'истории' 'история' 'ищу книгу' 'ищу фильм' 'карантин' 'книги'
 'коронавирус' 'кот' 'лига добра' 'лига юристов' 'любовь' 'люди' 'мат'
 'медицина' 'москва' 'музыка' 'мысли' 'негатив' 'новости' 'новый год'
 'общество' 'отношения' 'поиск' 'политика' 'помогите найти' 'помощь'
 'психология' 'работа' 'рассказ' 'реальная история из жизни'
 'родители и дети' 'россия' 'самоизоляция' 'санкт-петербург' 'семья'
 'случай из жизни' 'совет' 'сон' 'соседи' 'стихи' 'украина' 'фантастика'
 'фильмы' 'школа' 'юмор']


In [10]:
tag_distr = getworddict(getwordlist(data.tags))
tag_distr_formated = {}
for i in range(len(tag_distr)):
    tag_distr_formated[i] = round(tag_distr[Vec.get_feature_names_out()[i]] / sum(tag_distr.values()), 4)

In [11]:
print('Tags weights:')
print(tag_distr_formated)

Tags weights:
{0: 0.0073, 1: 0.0121, 2: 0.0075, 3: 0.0079, 4: 0.0085, 5: 0.0196, 6: 0.0077, 7: 0.0138, 8: 0.0089, 9: 0.0279, 10: 0.011, 11: 0.0221, 12: 0.0186, 13: 0.0122, 14: 0.0122, 15: 0.0086, 16: 0.0338, 17: 0.0067, 18: 0.008, 19: 0.014, 20: 0.0107, 21: 0.0454, 22: 0.0121, 23: 0.0142, 24: 0.0078, 25: 0.0149, 26: 0.0084, 27: 0.0549, 28: 0.0125, 29: 0.0098, 30: 0.0081, 31: 0.0093, 32: 0.0145, 33: 0.0152, 34: 0.0133, 35: 0.0237, 36: 0.0206, 37: 0.0071, 38: 0.0308, 39: 0.0118, 40: 0.054, 41: 0.0121, 42: 0.0297, 43: 0.0303, 44: 0.0315, 45: 0.0082, 46: 0.0277, 47: 0.0063, 48: 0.0075, 49: 0.0145, 50: 0.0122, 51: 0.0078, 52: 0.0067, 53: 0.0086, 54: 0.0274, 55: 0.0275, 56: 0.0111, 57: 0.0104, 58: 0.0144, 59: 0.0384}


In [12]:
print('Y shapes:')
print(f'  • Y train: {y_train.shape}')
print(f'  • Y validation: {y_val.shape}')
print(f'  • Y test: {y_test.shape}')

Y shapes:
  • Y train: (25209, 60)
  • Y validation: (2821, 60)
  • Y test: (3119, 60)


---

## 2.1. Training with bag-of-words embeddings:

In [13]:
save_models_path = module_path + '/models/svm/'

In [14]:
X_data = [' '.join(txt) for txt in data.text_markdown]

In [15]:
X_train = [' '.join(txt) for txt in data_train.text_markdown]
X_val = [' '.join(txt) for txt in data_val.text_markdown]
X_test = [' '.join(txt) for txt in data_test.text_markdown]

X_Vec = CountVectorizer(tokenizer = lambda x: x.split())

X_Vec.fit(X_train)
X_train = X_Vec.transform(X_train)
X_test = X_Vec.transform(X_test)
X_val = X_Vec.transform(X_val)

In [16]:
print("X BoW's shapes:")
print(f'   • X train shape: {X_train.shape}')
print(f'   • X val shape: {X_val.shape}')
print(f'   • X test shape: {X_test.shape}')

X BoW's shapes:
   • X train shape: (25209, 5899)
   • X val shape: (2821, 5899)
   • X test shape: (3119, 5899)


In [17]:
clf_ovr = OneVsRestClassifier(estimator=LinearSVC(penalty='l2',
                                                  loss='squared_hinge',
                                                  dual='auto',
                                                  C=1e3,
                                                  multi_class='ovr',
                                                  #class_weight=tag_distr_formated,
                                                  random_state=42),
                              n_jobs=-1)

In [18]:
if os.path.isfile(save_models_path + 'bow.joblib'):
    clf_ovr = load(save_models_path + 'bow.joblib')
else:
    clf_ovr.fit(X_train, y_train)
    dump(clf_ovr, save_models_path + 'bow.joblib')

In [19]:
bow_y_pred_val = clf_ovr.predict(X_val)

bow_df_val = data_val.copy()
bow_df_val['predicted_tags'] = Vec.inverse_transform(bow_y_pred_val)

In [20]:
bow_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],[помогите найти]
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],[работа]
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]","[девушки, мат, случай из жизни]"
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],"[лига юристов, новости]"
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, история, рассказ, фантастика]"


In [21]:
print('Metrics for Bag-of-Words:')
print(classification_report(y_val, bow_y_pred_val, zero_division=0))

Metrics for Bag-of-Words:
              precision    recall  f1-score   support

           0       0.31      0.40      0.35        25
           1       0.23      0.22      0.22        45
           2       0.48      0.33      0.39        40
           3       0.19      0.26      0.22        27
           4       0.42      0.41      0.42        27
           5       0.09      0.15      0.11        68
           6       0.36      0.33      0.34        40
           7       0.22      0.29      0.25        59
           8       0.08      0.10      0.09        42
           9       0.24      0.33      0.28       113
          10       0.24      0.26      0.25        50
          11       0.04      0.07      0.05        85
          12       0.06      0.11      0.08        72
          13       0.45      0.50      0.48        58
          14       0.02      0.02      0.02        52
          15       0.00      0.00      0.00        28
          16       0.12      0.16      0.14       135
 

Calculate `recall@k`:

In [22]:
K = 5
bow_recallk_mean, bow_recallk_med = recallk(bow_df_val.tags, bow_df_val.predicted_tags, k=K)

print(f'Mean recall@k for k={K} for Bag-of-Words: {bow_recallk_mean:.3f}')
#print(f'Median recall@k for k={K} for Bag-of-Words: {bow_recallk_med:.3f}')

Mean recall@k for k=5 for Bag-of-Words: 0.308


## 2.2 Training with IF-IDF embeddings:

In [23]:
X_train = [' '.join(txt) for txt in data_train.text_markdown]
X_val = [' '.join(txt) for txt in data_val.text_markdown]
X_test = [' '.join(txt) for txt in data_test.text_markdown]

Tfidf_Vec = TfidfVectorizer(tokenizer = lambda x: x.split())

Tfidf_Vec.fit(X_train)
X_train = Tfidf_Vec.transform(X_train)
X_test = Tfidf_Vec.transform(X_test)
X_val = Tfidf_Vec.transform(X_val)

c:\Users\decique\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [24]:
print("X TF-IDF's shapes:")
print(f'   • X train shape: {X_train.shape}')
print(f'   • X val shape: {X_val.shape}')
print(f'   • X test shape: {X_test.shape}')

X TF-IDF's shapes:
   • X train shape: (25209, 5899)
   • X val shape: (2821, 5899)
   • X test shape: (3119, 5899)


In [25]:
clf_ovr = OneVsRestClassifier(estimator=LinearSVC(penalty='l2',
                                                  loss='squared_hinge',
                                                  dual='auto',
                                                  C=1e3,
                                                  multi_class='ovr',
                                                  #class_weight=tag_distr_formated,
                                                  random_state=42),
                              n_jobs=-1)

In [26]:
if os.path.isfile(save_models_path + 'tf_idf.joblib'):
    clf_ovr = load(save_models_path + 'tf_idf.joblib')
else:
    clf_ovr.fit(X_train, y_train)
    dump(clf_ovr, save_models_path + 'tf_idf.joblib')

In [27]:
tf_idf_y_pred_val = clf_ovr.predict(X_val)

tf_idf_df_val = data_val.copy()
tf_idf_df_val['predicted_tags'] = Vec.inverse_transform(tf_idf_y_pred_val)

In [28]:
tf_idf_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],"[интересное, помогите найти]"
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],[работа]
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]",[мат]
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],"[лига юристов, новости]"
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, история, рассказ, фантастика]"


In [29]:
print('Metrics for TF-IDF:')
print(classification_report(y_val, tf_idf_y_pred_val, zero_division=0))

Metrics for TF-IDF:
              precision    recall  f1-score   support

           0       0.38      0.24      0.29        25
           1       0.30      0.24      0.27        45
           2       0.65      0.33      0.43        40
           3       0.37      0.26      0.30        27
           4       0.57      0.44      0.50        27
           5       0.11      0.13      0.12        68
           6       0.48      0.28      0.35        40
           7       0.26      0.25      0.26        59
           8       0.07      0.05      0.06        42
           9       0.28      0.33      0.30       113
          10       0.31      0.20      0.24        50
          11       0.03      0.05      0.04        85
          12       0.09      0.14      0.11        72
          13       0.62      0.48      0.54        58
          14       0.03      0.02      0.02        52
          15       0.00      0.00      0.00        28
          16       0.13      0.19      0.16       135
       

Calculate `recall@k`:

In [30]:
tf_idf_recallk_mean, tf_idf_recallk_med = recallk(tf_idf_df_val.tags, tf_idf_df_val.predicted_tags, k=K)

print(f'Mean recall@k for k={K} for TF-IDF: {tf_idf_recallk_mean:.3f}')
#print(f'Median recall@k for k={K} for TF-IDF: {tf_idf_recallk_med:.3f}')

Mean recall@k for k=5 for TF-IDF: 0.299


## 2.3. Training on rubert-tiny-v2 embeddings:

In [31]:
emb_paths = module_path + '/data/embeddings/rubert-tiny-v2/'

emb_pth = [emb_paths + 'texts.parquet']
emb = merge_dataset(emb_pth)

In [32]:
emb.head(3)

,id,embedding
0,2936217,"[0.3411005, -0.16877297, -0.3599054, 0.011505239, -0.19693527, 0.16206133, -0.62560713, -0.38459125, -0.08364315, -0.17384137,..."
1,6991412,"[0.3696494, 0.06409113, -0.62138826, -0.8906186, 0.08984075, 0.27482352, -0.31647494, -0.778525, -0.6068895, 0.42193377, -0.05..."
2,6991359,"[0.3278318, 0.08586374, -0.7452521, -0.2529353, -0.32851255, 0.5415312, -0.53284395, -1.2018805, 0.120118916, 0.15034527, -0.3..."


In [33]:
emb = emb[emb['id'].isin(data['id'])]

emb_train = emb[emb['id'].isin(data_train['id'])]
emb_val = emb[emb['id'].isin(data_val['id'])]
emb_test = emb[emb['id'].isin(data_test['id'])]

assert len(emb_train) == len(data_train), "Something went wrong!"
assert len(emb_val) == len(data_val), "Something went wrong!"
assert len(emb_test) == len(data_test), "Something went wrong!"

In [34]:
X_train_emb = [i for i in emb_train.embedding]
X_val_emb = [i for i in emb_val.embedding]
X_test_emb = [i for i in emb_test.embedding]

In [35]:
print("X embeddings shapes:")
print(f'   • X train shape: {np.shape(X_train_emb)}')
print(f'   • X val shape: {np.shape(X_val_emb)}')
print(f'   • X test shape: {np.shape(X_test_emb)}')

X embeddings shapes:
   • X train shape: (25209, 312)
   • X val shape: (2821, 312)
   • X test shape: (3119, 312)


In [36]:
clf_ovr = OneVsRestClassifier(estimator=LinearSVC(penalty='l2',
                                                  loss='squared_hinge',
                                                  dual='auto',
                                                  C=1e3,
                                                  multi_class='ovr',
                                                  #class_weight=tag_distr_formated,
                                                  random_state=42),
                              n_jobs=-1)

In [37]:
if os.path.isfile(save_models_path + 'rubert.joblib'):
    clf_ovr = load(save_models_path + 'rubert.joblib')
else:
    clf_ovr.fit(X_train_emb, y_train)
    dump(clf_ovr, save_models_path + 'rubert.joblib')

In [38]:
rubert_y_pred_val = clf_ovr.predict(X_val_emb)

rubert_df_val = data_val.copy()
rubert_df_val['predicted_tags'] = Vec.inverse_transform(rubert_y_pred_val)

In [39]:
rubert_df_val.head(5)

,id,text_markdown,tags,predicted_tags
421,6992880,"[популярный, пк, игра, создавать, устройство, проходить, картинка, память, оставаться, различный, создавать, написать, догадка...",[помогите найти],[игры]
432,6992917,"[профессия, оказываться, сопровождать, образование, курсы, обязанность, входить, хотеться, разговор, зажигать, свеча, оказыват...",[психология],[]
578,6994231,"[предыдущий, пост, бригада, график, переписывать, месяц, дата, окончание, поездка, графика, случаться, называть, двойной, поез...","[юмор, реальная история из жизни]",[]
591,6992488,"[широкий, хотеться, обращать, внимание, инцидент, связанный, дтп, участие, пожарный, двигаться, оперативный, вызов, включать, ...",[помощь],[]
807,7021892,"[история, скоро, девушка, упасть, дерево, парень, неловко, улыбаться, следовать, забывать, следить, окружение, темно, отзывать...","[рассказ, фантастика, мат]","[авторский рассказ, рассказ]"


In [40]:
print('Metrics for rubert embeddings:')
print(classification_report(y_val, rubert_y_pred_val, zero_division=0))

Metrics for rubert embeddings:
              precision    recall  f1-score   support

           0       0.35      0.28      0.31        25
           1       0.20      0.02      0.04        45
           2       1.00      0.05      0.10        40
           3       0.75      0.11      0.19        27
           4       0.45      0.19      0.26        27
           5       0.00      0.00      0.00        68
           6       0.50      0.07      0.13        40
           7       0.25      0.02      0.03        59
           8       1.00      0.05      0.09        42
           9       0.58      0.16      0.25       113
          10       0.75      0.06      0.11        50
          11       0.00      0.00      0.00        85
          12       0.00      0.00      0.00        72
          13       0.68      0.40      0.50        58
          14       0.00      0.00      0.00        52
          15       0.00      0.00      0.00        28
          16       0.00      0.00      0.00       

Calculate `recall@k`:

In [41]:
rubert_recallk_mean, rubert_recallk_med = recallk(rubert_df_val.tags, rubert_df_val.predicted_tags, k=K)

print(f'Mean recall@k for k={K} for model with rubert embeddings: {rubert_recallk_mean:.3f}')
#print(f'Median recall@k for k={K} for model with rubert embeddings:: {rubert_recallk_med:.3f}')

Mean recall@k for k=5 for model with rubert embeddings: 0.163


---
# 3. Results

As the result of training with different embeddings, we have the following:

In [42]:
print(f"Mean recall@k's for SVM with k = {K}:\n")
print(f'  • Recall@k for Bag-of-Words: {bow_recallk_mean:.4f}')
print(f'  • Recall@k for TF-IDF: {tf_idf_recallk_mean:.4f}')
print(f'  • Recall@k for rubert embeddings: {rubert_recallk_mean:.4f}')

#print(f"\nMedian recall@k's for SVM with k = {K}:\n")
#print(f'  • Recall@k for Bag-of-Words: {bow_recallk_med:3f}')
#print(f'  • Recall@k for TF-IDF: {tf_idf_recallk_med:3f}')
#print(f'  • Recall@k for rubert embeddings: {rubert_recallk_med:3f}')

Mean recall@k's for SVM with k = 5:

  • Recall@k for Bag-of-Words: 0.3080
  • Recall@k for TF-IDF: 0.2990
  • Recall@k for rubert embeddings: 0.1625
